<a href="https://colab.research.google.com/github/JulianLangberg/Bootcamp-DataScience/blob/main/NYC_vehicle_collisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit
!pip install pyngrok
!pip install geopandas
#!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 16.7 MB 324 kB/s 
     |████████████████████████████████| 6.3 MB 20.1 MB/s 


In [ ]:
from pyngrok import ngrok
import pandas as pd 

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import calendar
# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter



In [ ]:
tunnels = ngrok.get_tunnels()

In [ ]:
tunnels

[<NgrokTunnel: "https://39e4-34-86-250-156.ngrok.io" -> "http://localhost:80">,
 <NgrokTunnel: "http://39e4-34-86-250-156.ngrok.io" -> "http://localhost:80">]

In [ ]:
ngrok.set_auth_token('2DqrFm7dgxuB9GsRi3Fszz9sL4n_5jbYFP5ewghTKPE99gz4Z')

In [ ]:
public_url = ngrok.connect(port='8501')
public_url

<NgrokTunnel: "http://5d4f-34-71-226-168.ngrok.io" -> "http://localhost:80">

In [ ]:
%%writefile streamlit_app.py 

# Libraries
import streamlit as st 
import pandas as pd
from PIL import Image
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
### Librerias mapas

import geopandas as gpd



header1 = st.container()
#dataset = st.container()
mod1 = st.container()


# Empezamos con CSV.

file = '/content/drive/MyDrive/Colab Notebooks/Motor_Vehicle_Collisions-Crashes1.csv'
#file = 'Motor_Vehicle_Collisions-Crashes1.csv'
df3 = pd.read_csv(file)
df_p = df3

###########  VISUAL, PRESENTACION EQUIPO

with header1:
    st.title("Siniestralidad vial en NYC: análisis descriptivo y su impacto en la economía")
    image = Image.open('/content/drive/MyDrive/Colab Notebooks/images/portada_nyc_car_crash.jpg')

    st.image(image, caption='NYC - Car crash') 


    st.markdown("En la presente investigación se utilizará información oficial sobre siniestralidad vial recolectada desde https://opendata.cityofnewyork.us/, información climatológica del National Centers for Environmental Information en https://www.ncei.noaa.gov/, y los costos económicos generados surgen de investigaciones presentadas por el National Safety Counsil en https://injuryfacts.nsc.org/ ")
    st.markdown("Equipo de trabajo:")
    st.markdown("Arnone, Miguel")
    st.markdown("Langberg, Julian")
    st.markdown("Ojeda, Guillermo Agustín")
    st.markdown("Villarraga-Morales, Carolina")
with mod1:
    st.title("A total NYC, podemos apreciar una fuerte caída en la siniestralidad en los últimos dos año ")
    st.markdown("En el gráfico se observa como la cantidad de siniestros se reducen posterior al lockdown en el comienzo de la pandemia por Covid-19")

###### TRAIGO TABLAS, ETL  Y  GRAFICO INTRODUCTORIO 

df_p = df3

df_definitivo = df_p.groupby('crash_date').agg({'crash_date': ['count'], 'number_of_persons_injured': ['sum'], 'number_of_persons_killed': ['sum'] }).reset_index()
df_definitivo = df_definitivo.droplevel(level=1, axis=1)
df_definitivo.columns = ['crash_date','number_of_crashes','number_of_persons_injured','number_of_persons_killed']
df_definitivo.sort_values(by=['crash_date'])
df_definitivo['siniestros_media_movil']=savgol_filter(df_definitivo['number_of_crashes'], 51, 3)
df_definitivo['Lastimados_media_movil'] = savgol_filter(df_definitivo['number_of_persons_injured'], 51, 3)
df_definitivo['Muertos_media_movil'] = savgol_filter(df_definitivo['number_of_persons_killed'], 51, 3)


#### Grafico evolutivo

fig = px.line(df_definitivo, x = df_definitivo['crash_date'],
                y=df_definitivo['siniestros_media_movil'], 
                title='Evolución siniestros NYC', width=800, height=550,
                labels= {'crash_date':'Fechas','value':'Cantidad'})

fig.add_trace(
    go.Scatter(x=list(df_definitivo.crash_date), y=list(df_definitivo['siniestros_media_movil'])
                , name='Cantidad de siniestros_media_móvil'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

st.plotly_chart(fig, use_container_width=True)

## Grafico evolutivo

st.markdown("Haciendo foco en la cantidad de heridos, podemos encontrar un patrón de máximos y mínimos")

fig = px.line(df_definitivo,x = df_definitivo['crash_date'],
                y=['Lastimados_media_movil','Muertos_media_movil'], 
                title='Evolución siniestros NYC', width=800, height=550,
                labels= {'crash_date':'Fechas','value':'Cantidad'})

fig.add_trace(
    go.Scatter(x=list(df_definitivo.crash_date), y=list(df_definitivo['Lastimados_media_movil'])
                , name='Cantidad de personas lastimadas'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

st.plotly_chart(fig, use_container_width=True)


#st.title("A total NYC, podemos apreciar una fuerte caída en la siniestralidad en los últimos dos año ")
st.markdown("Al profundizar en estos valores por temporada, observamos que los picos de heridos por siniestros de tránsito ocurren en verano")

#### ETL PARA GRAFICO DE TEMPORADASS

def MonthToSeason(x):   
    global season
    if x == 3 or x == 4 or x == 5:
         season = "spring"
    elif x == 6 or x == 7 or x == 8:
         season = "summer"
    elif x == 9 or x == 10 or x == 11:
         season = "autumn"
    elif x == 12 or x == 0 or x == 1:
         season = "winter"
    else:
         season = np.nan 
    return season

df_definitivo['Season'] = pd.to_datetime(df_definitivo['crash_date']).dt.month.apply(lambda x : MonthToSeason(x))
df_season_siniestros= df_definitivo.groupby('Season')[['number_of_crashes']].sum().reset_index()
fig = px.bar(df_season_siniestros, x="Season", y=["number_of_crashes"], title="Siniestros por temporada")

st.plotly_chart(fig, use_container_width=True)



st.markdown("De similar forma, se observa que los días viernes también existen mayor cantidad de siniestros")



#### ETL PARA GRAFICO DE DIAS DE SEMANA

df_definitivo['week_day']=pd.to_datetime(df_definitivo['crash_date']).dt.dayofweek

df_semana_siniestros= df_definitivo.groupby('week_day')[['number_of_crashes']].sum().reset_index()

df_semana_siniestros['week_day'].replace([0,1,2,3,4,5,6],['lunes','martes','miércoles','jueves','viernes','sábado','domingo'],inplace=True)

fig = px.bar(df_semana_siniestros, x="week_day", y=["number_of_crashes"], title="Siniestros por día de la semana")

st.plotly_chart(fig, use_container_width=True)


###### gráfico de mapas 1ero ETL

df_p.drop(df_p[df_p.collision_id.isnull()].index,inplace=True)
df_p['collision_id']=df_p['collision_id'].astype(int)
df_pc=df_p.set_index('collision_id')
df_pc.head(1)
df_bor=df_pc.loc[:,['borough','zip_code', 'latitude','longitude','location.latitude','location.longitude']]
lat_0=df_bor[df_bor.latitude==df_bor.latitude.isna()]
df_geo= df_pc.copy() #traer los datos con al menos borough
df_geo.drop(df_bor[df_bor.notnull().sum(axis=1)==0].index,inplace=True) #borrando latlon NaN 
df_geo.drop(lat_0[lat_0.notnull().sum(axis=1)==4].index,inplace=True) #borrando latlon 0 0
#
df_geo['year']=pd.to_datetime(df_geo.crash_date).dt.year
df_ptos = gpd.GeoDataFrame(df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)) #esto crea una nueva columna llamada 
bor_year=df_ptos.groupby(['borough', 'year']).sum().reset_index()
bor_year.drop(columns=['latitude','longitude','location.latitude','location.longitude'], inplace=True)
bor_year.columns.to_list()

### GRAFICO MAPAAAA

nyc = gpd.read_file(gpd.datasets.get_path('nybb'))
#Rename the column to boroname, so that we can join the data to it on a common field
nyc.rename(columns={'BoroName':'borough'}, inplace=True)
nyc['borough']=nyc['borough'].str.upper()##para poder hacer merge con bor_year ya que tiene values en mayusculas
bc_geo = nyc.merge(bor_year, on='borough')


fig,ax = plt.subplots(1,1, figsize=(10,10))
bc_geo.plot(column='number_of_persons_injured', cmap='viridis_r', alpha=.5, ax=ax, legend=True)
bc_geo.apply(lambda x: ax.annotate(text=x.borough, color='black', xy=x.geometry.centroid.coords[0],ha='center'), axis=1)

bc_geo.plot(ax=ax, color='red')
st.pyplot()




plt.title("Number of persons injuered by NYC Borough")
plt.axis('off')













##### ETL PARA CALCULADO KPI's
####Si el archivo es el descargado como csv, se debe hacer esto, sino comenta el codigo el cambio a datetime
data = df3


data.columns = map(str.lower, data.columns) #cambiar nombres de las columnas por minusculas
d2=data.columns.str.replace(' ', '_')#reemplazar espacios por guion bajo, es un index type
d2=d2.to_list()
data.columns=d2 #cambiar por valores de la lista
#
data['crash_date']=pd.to_datetime(data['crash_date'])
data = data.sort_values('crash_date',ascending=True)

################DASHBOARD#############################
#Esto solo es por joder, se ve lindi si le hacemos click en la seccion de menu


# dashboard title
st.title("Real-Time Data Science Dashboard")
st.markdown("Con las siguientes herramientas, será posible dimensionar la cantidad de personas directamente afectadas por los siniestros en un determinado período, y preparar indicadores que nos permitan posteriormente calcular cuantas personas se verían beneficiadas por politicas precautorias en el sector de Seguridad Vial.")
#Seleccion de fecha
start_date = st.date_input("Seleccionar fecha de inicio",min_value = datetime.date(2021, 8, 26), max_value= datetime.date.today(), value=datetime.date.today())
end_date = st.date_input("Seleccionar último dia de análisis",min_value = datetime.date(2012, 7, 1), max_value= datetime.date.today(), value=datetime.date.today()) #ver si esta actualizando siempre!
filter=data.copy() #voy a hacer un campo de tipo de fecha para poder hacer la mascara, prefiero hacerlo con copia
filter['crash_date']=filter['crash_date'].dt.date
filter=filter[(filter['crash_date']>= (start_date)) & (filter['crash_date']<=(end_date))]

#Calculo de dias/anhos/deltas
deltat=(end_date-start_date)
dias=deltat.days
anhos=round(deltat.days/365,1)
future=datetime.date.today()+deltat


#Para el analisis por ahora solo se tiene en cuenta numero de accidendentes, personas heridas y muertas
r = filter.groupby('crash_date').agg({'crash_date': ['count'], 'number_of_persons_injured': ['sum'], 'number_of_persons_killed': ['sum'] }).reset_index()
r.columns = ['crash_date','number_of_crashes','number_of_persons_injured','number_of_persons_killed']


d=st.markdown(f'Tiempo total en **dias**= **_{dias}_**')
a=st.markdown(f'Tiempo total en **años**= **_{anhos}_**')
# COSTOS ECONÓMICOS
death=1750000
injured=26550
car_damage=4700    

#resumen de los datos en forma de metricas
accidentes=r.number_of_crashes.sum()
heridos=r.number_of_persons_injured.sum()
muertos=r.number_of_persons_killed.sum()
costo_economico=round((car_damage*accidentes+injured*heridos+muertos*death)/1000000,2)


col1, col2, col3, col4 = st.columns(4)
col1.metric("Accidentes", value=accidentes)
col2.metric("Heridos", value=heridos)
col3.metric("Muertos",  value=muertos)
col4.metric('Costo económico ($Mill.)', value=costo_economico)

#####"Calculadora", intento de KPI v0.0.1
##Reducir x% los choques en y anhos respecto a z cantidad de tiempo

st.header(f"KPIs objetivo sobre el periodo de tiempo seleccionado:")

porcentaje=st.number_input('% deseado de reduccion de accidentes',min_value=0.0, max_value=100.0, value=0.0)

calculo=st.button('Click para calcular')

if calculo:
    st.markdown(f'Un {porcentaje} % representa una _disminucion_ de {accidentes*porcentaje/100} accidentes.')
    st.markdown(f'Esta reducción de accidentes tendría un impacto positívo en la economía de {round(costo_economico*porcentaje/100,1)} millones de USD.')
    st.markdown(f'Esta meta tendria que alcanzarse en **{dias}** dias, {future}')
    
    
    st.metric("Accidentes menos por dia", value=round((accidentes*porcentaje/100)/dias,2), delta= f'-{porcentaje}%' )
image = Image.open('/content/drive/MyDrive/Colab Notebooks/images/Conclusiones_Iniciales_NYC.png')

st.image(image, caption='NYC - Car crash') 

st.subheader( '- La pandemia tuvo como resultado colateral una significativa caída en los accidentes de tránsito. La principal hipótesis a validar es si esta caída post-Lockdown es generada por menos personas movilizandose a sus oficinas de trabajo. Una posible forma de validar esto es observando si esta caída es mayor en días hábiles que fines de semana, pre y post pandemia.')
st.subheader( '- Se observa mayor siniestralidad en verano, y los días viernes. Una linea de investigación que se profundizará es la correlación de estos valores con la conducción "DUI".')
st.subheader( '- El impacto económico por la siniestralidad vial toma valores multimillonarios. Es necesario conocer políticas públicas pasadas hacia el sector, para identificar el impacto que generan y medir posibles beneficios de estas.')





Overwriting streamlit_app.py


In [ ]:
%%writefile streamlit_app.py 

# Libraries
import streamlit as st 
import pandas as pd
from PIL import Image
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.signal import savgol_filter


header1 = st.container()
#dataset = st.container()
mod1 = st.container()


# Empezamos con CSV.

file = '/content/drive/MyDrive/Colab Notebooks/Motor_Vehicle_Collisions-Crashes1.csv'
#file = 'Motor_Vehicle_Collisions-Crashes1.csv'
df3 = pd.read_csv(file)
df_p = df3

###########  VISUAL, PRESENTACION EQUIPO

with header1:
    st.title("Siniestralidad vial en NYC: análisis descriptivo y su impacto en la economía")
    image = Image.open('/content/drive/MyDrive/Colab Notebooks/images/portada_nyc_car_crash.jpg')

    st.image(image, caption='NYC - Car crash') 


    st.markdown("En la presente investigación se utilizará información oficial sobre siniestralidad vial recolectada desde https://opendata.cityofnewyork.us/, información climatológica del National Centers for Environmental Information en https://www.ncei.noaa.gov/, y los costos económicos generados surgen de investigaciones presentadas por el National Safety Counsil en https://injuryfacts.nsc.org/ ")
    st.markdown("Equipo de trabajo:")
    st.markdown("Arnone, Miguel")
    st.markdown("Langberg, Julian")
    st.markdown("Ojeda, Guillermo Agustín")
    st.markdown("Villarraga-Morales, Carolina")
with mod1:
    st.title("A total NYC, podemos apreciar una fuerte caída en la siniestralidad en los últimos dos año ")
    st.markdown("En el gráfico se observa como la cantidad de siniestros se reducen posterior al lockdown en el comienzo de la pandemia por Covid-19")

###### TRAIGO TABLAS, ETL  Y  GRAFICO INTRODUCTORIO 

df_p = df3

df_definitivo = df_p.groupby('crash_date').agg({'crash_date': ['count'], 'number_of_persons_injured': ['sum'], 'number_of_persons_killed': ['sum'] }).reset_index()
df_definitivo = df_definitivo.droplevel(level=1, axis=1)
df_definitivo.columns = ['crash_date','number_of_crashes','number_of_persons_injured','number_of_persons_killed']
df_definitivo.sort_values(by=['crash_date'])
df_definitivo['siniestros_media_movil']=savgol_filter(df_definitivo['number_of_crashes'], 51, 3)
df_definitivo['Lastimados_media_movil'] = savgol_filter(df_definitivo['number_of_persons_injured'], 51, 3)
df_definitivo['Muertos_media_movil'] = savgol_filter(df_definitivo['number_of_persons_killed'], 51, 3)


#### Grafico evolutivo

fig = px.line(df_definitivo, x = df_definitivo['crash_date'],
                y=df_definitivo['siniestros_media_movil'], 
                title='Evolución siniestros NYC', width=800, height=550,
                labels= {'crash_date':'Fechas','value':'Cantidad'})

fig.add_trace(
    go.Scatter(x=list(df_definitivo.crash_date), y=list(df_definitivo['siniestros_media_movil'])
                , name='Cantidad de siniestros_media_móvil'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

st.plotly_chart(fig, use_container_width=True)

## Grafico evolutivo

st.markdown("Haciendo foco en la cantidad de heridos, podemos encontrar un patrón de máximos y mínimos")

fig = px.line(df_definitivo,x = df_definitivo['crash_date'],
                y=['Lastimados_media_movil','Muertos_media_movil'], 
                title='Evolución siniestros NYC', width=800, height=550,
                labels= {'crash_date':'Fechas','value':'Cantidad'})

fig.add_trace(
    go.Scatter(x=list(df_definitivo.crash_date), y=list(df_definitivo['Lastimados_media_movil'])
                , name='Cantidad de personas lastimadas'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

st.plotly_chart(fig, use_container_width=True)


#st.title("A total NYC, podemos apreciar una fuerte caída en la siniestralidad en los últimos dos año ")
st.markdown("Al profundizar en estos valores por temporada, observamos que los picos de heridos por siniestros de tránsito ocurren en verano")

#### ETL PARA GRAFICO DE TEMPORADASS

def MonthToSeason(x):   
    global season
    if x == 3 or x == 4 or x == 5:
         season = "spring"
    elif x == 6 or x == 7 or x == 8:
         season = "summer"
    elif x == 9 or x == 10 or x == 11:
         season = "autumn"
    elif x == 12 or x == 0 or x == 1:
         season = "winter"
    else:
         season = np.nan 
    return season

df_definitivo['Season'] = pd.to_datetime(df_definitivo['crash_date']).dt.month.apply(lambda x : MonthToSeason(x))
df_season_siniestros= df_definitivo.groupby('Season')[['number_of_crashes']].sum().reset_index()
fig = px.bar(df_season_siniestros, x="Season", y=["number_of_crashes"], title="Siniestros por temporada")

st.plotly_chart(fig, use_container_width=True)



st.markdown("De similar forma, se observa que los días viernes también existen mayor cantidad de siniestros")



#### ETL PARA GRAFICO DE DIAS DE SEMANA

df_definitivo['week_day']=pd.to_datetime(df_definitivo['crash_date']).dt.dayofweek

df_semana_siniestros= df_definitivo.groupby('week_day')[['number_of_crashes']].sum().reset_index()

df_semana_siniestros['week_day'].replace([0,1,2,3,4,5,6],['lunes','martes','miércoles','jueves','viernes','sábado','domingo'],inplace=True)

fig = px.bar(df_semana_siniestros, x="week_day", y=["number_of_crashes"], title="Siniestros por día de la semana")

st.plotly_chart(fig, use_container_width=True)




##### ETL PARA CALCULADO KPI's
####Si el archivo es el descargado como csv, se debe hacer esto, sino comenta el codigo el cambio a datetime
data = df3


data.columns = map(str.lower, data.columns) #cambiar nombres de las columnas por minusculas
d2=data.columns.str.replace(' ', '_')#reemplazar espacios por guion bajo, es un index type
d2=d2.to_list()
data.columns=d2 #cambiar por valores de la lista
#
data['crash_date']=pd.to_datetime(data['crash_date'])
data = data.sort_values('crash_date',ascending=True)

################DASHBOARD#############################
#Esto solo es por joder, se ve lindi si le hacemos click en la seccion de menu


# dashboard title
st.title("Real-Time Data Science Dashboard")
st.markdown("Con las siguientes herramientas, será posible dimensionar la cantidad de personas directamente afectadas por los siniestros en un determinado período, y preparar indicadores que nos permitan posteriormente calcular cuantas personas se verían beneficiadas por politicas precautorias en el sector de Seguridad Vial.")
#Seleccion de fecha
start_date = st.date_input("Seleccionar fecha de inicio",min_value = datetime.date(2021, 8, 26), max_value= datetime.date.today(), value=datetime.date.today())
end_date = st.date_input("Seleccionar último dia de análisis",min_value = datetime.date(2012, 7, 1), max_value= datetime.date.today(), value=datetime.date.today()) #ver si esta actualizando siempre!
filter=data.copy() #voy a hacer un campo de tipo de fecha para poder hacer la mascara, prefiero hacerlo con copia
filter['crash_date']=filter['crash_date'].dt.date
filter=filter[(filter['crash_date']>= (start_date)) & (filter['crash_date']<=(end_date))]

#Calculo de dias/anhos/deltas
deltat=(end_date-start_date)
dias=deltat.days
anhos=round(deltat.days/365,1)
future=datetime.date.today()+deltat


#Para el analisis por ahora solo se tiene en cuenta numero de accidendentes, personas heridas y muertas
r = filter.groupby('crash_date').agg({'crash_date': ['count'], 'number_of_persons_injured': ['sum'], 'number_of_persons_killed': ['sum'] }).reset_index()
r.columns = ['crash_date','number_of_crashes','number_of_persons_injured','number_of_persons_killed']


d=st.markdown(f'Tiempo total en **dias**= **_{dias}_**')
a=st.markdown(f'Tiempo total en **años**= **_{anhos}_**')
# COSTOS ECONÓMICOS
death=1750000
injured=26550
car_damage=4700    

#resumen de los datos en forma de metricas
accidentes=r.number_of_crashes.sum()
heridos=r.number_of_persons_injured.sum()
muertos=r.number_of_persons_killed.sum()
costo_economico=round((car_damage*accidentes+injured*heridos+muertos*death)/1000000,2)


col1, col2, col3, col4 = st.columns(4)
col1.metric("Accidentes", value=accidentes)
col2.metric("Heridos", value=heridos)
col3.metric("Muertos",  value=muertos)
col4.metric('Costo económico ($Mill.)', value=costo_economico)

#####"Calculadora", intento de KPI v0.0.1
##Reducir x% los choques en y anhos respecto a z cantidad de tiempo

st.header(f"KPIs objetivo sobre el periodo de tiempo seleccionado:")

porcentaje=st.number_input('% deseado de reduccion de accidentes',min_value=0.0, max_value=100.0, value=0.0)

calculo=st.button('Click para calcular')

if calculo:
    st.markdown(f'Un {porcentaje} % representa una _disminucion_ de {accidentes*porcentaje/100} accidentes.')
    st.markdown(f'Esta reducción de accidentes tendría un impacto positívo en la economía de {round(costo_economico*porcentaje/100,1)} millones de USD.')
    st.markdown(f'Esta meta tendria que alcanzarse en **{dias}** dias, {future}')
    
    
    st.metric("Accidentes menos por dia", value=round((accidentes*porcentaje/100)/dias,2), delta= f'-{porcentaje}%' )
image = Image.open('/content/drive/MyDrive/Colab Notebooks/images/Conclusiones_Iniciales_NYC.png')

st.image(image, caption='NYC - Car crash') 

st.subheader( '- La pandemia tuvo como resultado colateral una significativa caída en los accidentes de tránsito. La principal hipótesis a validar es si esta caída post-Lockdown es generada por menos personas movilizandose a sus oficinas de trabajo. Una posible forma de validar esto es observando si esta caída es mayor en días hábiles que fines de semana, pre y post pandemia.')
st.subheader( '- Se observa mayor siniestralidad en verano, y los días viernes. Una linea de investigación que se profundizará es la correlación de estos valores con la conducción "DUI".')
st.subheader( '- El impacto económico por la siniestralidad vial toma valores multimillonarios. Es necesario conocer políticas públicas pasadas hacia el sector, para identificar el impacto que generan y medir posibles beneficios de estas.')




Overwriting streamlit_app.py


In [ ]:
!streamlit run /content/streamlit_app.py & npx localtunnel --port 8501

2022-08-26 18:49:38.430 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.71.226.168:8501

npx: installed 22 in 4.142s
your url is: https://major-nights-spend-34-71-226-168.loca.lt
/usr/local/lib/python3.7/dist-packages/streamlit/runtime/scriptrunner/script_runner.py:556: DtypeWarning: Columns (18,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, module.__dict__)
/usr/local/lib/python3.7/dist-packages/streamlit/runtime/scriptrunner/script_runner.py:556: DtypeWarning:

Columns (18,24) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = '/content/drive/MyDrive/Colab Notebooks/Motor_Vehicle_Collisions-Crashes1.csv'

df3 = pd.read_csv(file)
df_p = df3

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
